In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pytorch_lightning as pl
import torch

from model.CringeLDM import CringeLDM
from PIL import Image

from utils import convert_to_rgb, import_image_from_path

In [ ]:
from data.unsplashlite import UnsplashLiteDataset
import torch.utils.data
from torch.utils.data import DataLoader
import pytorch_lightning as pl

os.environ['CUDA_VISIBLE_DEVICES'] ='0'

# data
dataset = UnsplashLiteDataset(root_dir='/mnt/e/Source/unsplash-lite-corpus-preprocess/db')
training_set, validation_set = torch.utils.data.random_split(dataset, [int(len(dataset)*0.8), int(len(dataset)*0.2)])

train_loader = DataLoader(training_set, batch_size=1)
val_loader = DataLoader(validation_set, batch_size=1)

# model
model = CringeLDM().to("cuda:0" if torch.cuda.is_available() else "cpu")

# training
trainer = pl.Trainer(accelerator='gpu', precision=16, limit_train_batches=0.5)
trainer.fit(model, train_loader, val_loader)


In [ ]:
# Import an image
img = import_image_from_path()

# Load the image
res = model.forward_with_q(query="I am a cat", x=img, steps=1);

def convert_to_rgb2(x):
    x = x.squeeze(0)
    x = x.permute(1, 2, 0)
    x = x.detach().to("cpu").numpy()
    x = (x - x.min()) / (x.max() - x.min())
    x = (x * 255).astype(np.uint8)
    return x

# Convert the image to RGB
res = convert_to_rgb2(res)

# Show the image
plt.imshow(res)

In [ ]:
torch.save(model.state_dict(), "checkpoints/model.pt")

In [ ]:

import torch
import torch.nn as nn
from model.cringe.unet import ConvBlock, UpBlock

# Define a convolutional layer
conv = UpBlock(3,3)

# Define a transpose convolutional layer
transpose_conv = nn.ConvTranspose2d(3, 3, kernel_size=3, stride=3)

# Generate some random input data
x = torch.randn(1, 3, 10, 10)
# Generate some random input data
x2 = torch.randn(1, 3, 20, 20)

# Apply the convolutional layer to the input data
y = conv(x, x2)

# Upsample the output of the convolutional layer by a factor of 2
z = transpose_conv(y)

print(f'Original size: {x.size()}')
print(f'Convolutional layer output size: {y.size()}')
print(f'Upsampled output size: {z.size()}')

In [2]:
from utils import *

img = import_image_from_path()
img = convert_to_rgb(img)
export_image_to_png(img, "test.png")